# JULIA MPI First Example: pi computaton

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [1]:
Pkg.add("MPI")

INFO: Cloning cache of MPI from https://github.com/JuliaParallel/MPI.jl.git
INFO: Installing MPI v0.6.0
INFO: Building MPI
INFO: Attempting to create directory /Users/alanedelman/.julia/v0.6/MPI/deps/build
INFO: Changing directory to /Users/alanedelman/.julia/v0.6/MPI/deps/build
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of MPI
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
using MPI

INFO: Precompiling module MPI.

Use "primitive type CComm 32 end" instead.


In [3]:
m = MPIManager(np=4)

MPI.MPIManager(np=4,launched=false,mode=MPI_ON_WORKERS)

In [4]:
addprocs(m)
#@mpi_do m comm = MPI.COMM_WORLD

4-element Array{Int64,1}:
 2
 3
 4
 5

In [5]:
@mpi_do m comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
 @mpi_do m n = rand()

In [6]:
# Let's see if the processors got it
@mpi_do m println(n)

	From worker 3:	0.8374721520098549
	From worker 4:	0.2988396507609099
	From worker 2:	0.1448924739355686
	From worker 5:	0.9784654722267607


In [7]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

	From worker 4:	2
	From worker 3:	1
	From worker 2:	0
	From worker 5:	3


In [8]:
# Get the number of processors
@mpi_do m np = MPI.Comm_size(comm)
@mpi_do m println(np)

	From worker 5:	4
	From worker 3:	4
	From worker 4:	4
	From worker 2:	4


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [9]:
using Interact

In [10]:
@time @mpi_do m begin
    n = 50_000_000
    comm = MPI.COMM_WORLD
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n 
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    if myid==0
        println(our_π - π) 
    end
end

	From worker 2:	1.1146639167236572e-13
  5.481203 seconds (4.46 k allocations: 287.995 KiB)


In [11]:
[( @fetchfrom i π-4*mypi, π  )for i in workers()]

4-element Array{Tuple{Float64,Irrational{:π}},1}:
 (-6.0e-8, π = 3.1415926535897...)    
 (-2.00002e-8, π = 3.1415926535897...)
 (1.99998e-8, π = 3.1415926535897...) 
 (5.99999e-8, π = 3.1415926535897...) 

In [12]:
function f_serial()
    n = 50_000_000
    h = 1/n
    our_π = 0.0
    for i = 0:h:1
        our_π += 4/(1 + i^2)
    end
    our_π*h
end

function f_serial2(n)
    our_π = 0.0
    for i = 1:n
        x = (i - 0.5)/n
        our_π += 4/(1 + x^2)
    end
    our_π/n
end

f_serial2 (generic function with 1 method)

In [13]:
f_serial() #warmup
f_serial()
f_serial2(50_000_000) #warmup
@time f_serial2(50_000_000)

  0.407441 seconds (5 allocations: 176 bytes)


3.1415926535895617

In [14]:
function f_par(n)

 @mpi_do m begin
    comm = MPI.COMM_WORLD
        
       
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : $n 
        x = (i - .5)/$n 
        s += 4/(1 + x^2) 
    end
    mypi = s/$n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    #if myid==0
     #   println(our_π - π) 
   # end
end
@fetchfrom 2 our_π   
end

f_par (generic function with 1 method)

In [15]:
@mpi_do m function _pi_sum_par(n)
    comm = MPI.COMM_WORLD

    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    return our_π
end
function f_par2(n)
    @mpi_do m _pi_sum_par($n)
    @fetchfrom 2 our_π
end
f_par(50_000_000) #warmup
f_par(50_000_000)
f_par2(50_000_000) #warmup
@time f_par2(50_000_000)

  0.114711 seconds (565 allocations: 39.344 KiB)


3.1415926535899046

In [16]:
π

π = 3.1415926535897...

In [17]:
[f_par(10^k) for k=3:9]-π

7-element Array{Float64,1}:
  8.33333e-8 
  8.33331e-10
  8.32312e-12
  1.10134e-13
 -1.07025e-13
  4.23661e-13
 -2.4869e-14 

In [18]:
using DistributedArrays

INFO: Recompiling stale cache file /Users/alanedelman/.julia/lib/v0.6/DistributedArrays.ji for module DistributedArrays.


In [20]:
Pkg.add("Elemental")

INFO: Package Elemental is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Elemental
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [21]:
using Elemental

In [22]:
A = drandn(1000, 800);

In [25]:
svdvals(A)

LoadError: [91mMethodError: no method matching svdvals!(::DistributedArrays.DArray{Float64,2,Array{Float64,2}})[0m
Closest candidates are:
  svdvals!([91m::Union{Base.ReshapedArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:DenseArray, DenseArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2}, SubArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2,A,I,L} where L} where I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex, Int64, Range{Int64}},N} where N} where A<:Union{Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:DenseArray where N where T, DenseArray}[39m) where T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64} at linalg/svd.jl:130
  svdvals!([91m::Union{Base.ReshapedArray{T<:Union{Complex{Float32}, Complex{Flo